In [1]:
from common_functions import ret_metabase, google_sheets, upload_dataframe_to_snowflake, snowflake_query, upload_dataframe_to_pg, dwh_query, get_secret
import pandas as pd
from datetime import datetime, timedelta
import datetime as dt
import time
import os
import boto3
import base64
from botocore.exceptions import ClientError
import json
import requests
from pathlib import Path
from io import StringIO
import pandas as pd
# import psycopg2
import numpy as np
import gspread
import sqlalchemy
import pytz
import psycopg2

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [3]:
campaign_offers = dwh_query("egypt", "select * from materialized_views.offer_tool_logs", "read")
campaign_offers

/home/ec2-user/service_account_key.json


,id,campaign_id,supplier_id,retailer_list,offer_type,status,start_at,end_at
0,1,11,NaN,"[42031, 770467, 390843, 471035, 21221, 373805,...",SKU_discounts,Pending,2025-09-15 16:12:05,2025-09-16 02:12:05
1,2,11,153.0,"[42031, 770467, 390843, 471035, 21221, 373805,...",SKU_discounts,Approved,2025-09-15 17:05:20,2025-09-16 03:05:20
2,3,12,153.0,"[42031, 770467, 390843, 471035, 21221, 373805,...",SKU_discounts,Approved,2025-09-15 17:20:07,2025-09-16 03:20:07
3,4,13,36.0,"[42031, 187880, 152179, 220400, 21221, 600256,...",offers,Pending,2025-09-17 12:00:00,2025-09-17 22:00:00
4,5,13,36.0,"[42031, 187880, 152179, 220400, 21221, 600256,...",offers,Pending,2025-09-17 12:00:00,2025-09-17 22:00:00
5,6,14,20.0,"[189262, 600256, 42031, 40069, 652631, 21221, ...",offers,Pending,2025-09-17 12:00:00,2025-09-17 22:00:00
6,7,15,20.0,"[42031, 89747, 189262, 180832, 21221, 304614, ...",SKU_discounts,Pending,2025-09-17 12:00:00,2025-09-17 22:00:00
7,8,16,376.0,"[42031, 258853, 456325, 21221, 205682, 787231,...",SKU_discounts,Pending,2025-09-17 14:10:28,2025-09-18 00:10:28
8,9,17,739.0,"[42031, 69636, 45131, 215258, 463672, 673304, ...",SKU_discounts,Pending,2025-09-18 12:00:00,2025-09-18 22:00:00
9,11,19,110.0,"[42031, 457975, 313057, 21221, 156922, 36959, ...",SKU_discounts,Pending,2025-09-18 13:43:19,2025-09-18 23:43:19


In [13]:
import pandas as pd
campaign_offers = campaign_offers[campaign_offers['offer_type'] == 'offers']
# Make sure end_at is a datetime column
campaign['end_at'] = pd.to_datetime(campaign['end_at'])

# Get current datetime
now = pd.Timestamp.now()

# Filter rows where end_at is in the future
campaign_ongoing = campaign[campaign['end_at'] > now]
campaign_ongoing


,id,campaign_id,supplier_id,retailer_list,offer_type,status,start_at,end_at
24,25,33,36.0,"[669974, 362245, 34719, 154218, 20102, 697332,...",SKU_discounts,Pending,2025-10-20 12:00:00,2025-10-20 22:00:00


In [8]:
campaign_ids = campaign_ongoing['campaign_id'].tolist()
print(campaign_ids)


[33]


In [9]:
offers_id = dwh_query("egypt", "select * from materialized_views.offer_tool_logs", "read")
offers_id

/home/ec2-user/service_account_key.json


,id,campaign_id,supplier_id,retailer_list,offer_type,status,start_at,end_at
0,1,11,NaN,"[42031, 770467, 390843, 471035, 21221, 373805,...",SKU_discounts,Pending,2025-09-15 16:12:05,2025-09-16 02:12:05
1,2,11,153.0,"[42031, 770467, 390843, 471035, 21221, 373805,...",SKU_discounts,Approved,2025-09-15 17:05:20,2025-09-16 03:05:20
2,3,12,153.0,"[42031, 770467, 390843, 471035, 21221, 373805,...",SKU_discounts,Approved,2025-09-15 17:20:07,2025-09-16 03:20:07
3,4,13,36.0,"[42031, 187880, 152179, 220400, 21221, 600256,...",offers,Pending,2025-09-17 12:00:00,2025-09-17 22:00:00
4,5,13,36.0,"[42031, 187880, 152179, 220400, 21221, 600256,...",offers,Pending,2025-09-17 12:00:00,2025-09-17 22:00:00
5,6,14,20.0,"[189262, 600256, 42031, 40069, 652631, 21221, ...",offers,Pending,2025-09-17 12:00:00,2025-09-17 22:00:00
6,7,15,20.0,"[42031, 89747, 189262, 180832, 21221, 304614, ...",SKU_discounts,Pending,2025-09-17 12:00:00,2025-09-17 22:00:00
7,8,16,376.0,"[42031, 258853, 456325, 21221, 205682, 787231,...",SKU_discounts,Pending,2025-09-17 14:10:28,2025-09-18 00:10:28
8,9,17,739.0,"[42031, 69636, 45131, 215258, 463672, 673304, ...",SKU_discounts,Pending,2025-09-18 12:00:00,2025-09-18 22:00:00
9,11,19,110.0,"[42031, 457975, 313057, 21221, 156922, 36959, ...",SKU_discounts,Pending,2025-09-18 13:43:19,2025-09-18 23:43:19


In [10]:
offers_filtered = offers_id[offers_id['campaign_id'].isin(campaign_ids)]
offers_filtered

,id,campaign_id,supplier_id,retailer_list,offer_type,status,start_at,end_at
24,25,33,36.0,"[669974, 362245, 34719, 154218, 20102, 697332,...",SKU_discounts,Pending,2025-10-20 12:00:00,2025-10-20 22:00:00


In [12]:
offers_filtered['end_at'] = pd.to_datetime(offers_filtered['end_at'])

# Get current datetime
now = pd.Timestamp.now()

# Filter rows where end_at is in the future
offers_ended = offers_filtered[offers_filtered['end_at'] < now]
offers_ended

/tmp/ipykernel_21497/215201034.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  offers_filtered['end_at'] = pd.to_datetime(offers_filtered['end_at'])


,id,campaign_id,supplier_id,retailer_list,offer_type,status,start_at,end_at


In [14]:
# Extract the values from offers_ended DataFrame first
if not offers_ended.empty:
    supplier_id_value = offers_ended['supplier_id'].iloc[0]
    retailer_id_value = offers_ended['retailer_list'].iloc[0]
    print(f"Supplier ID: {supplier_id_value}")
    print(f"Retailer ID: {retailer_id_value}")
else:
    print("offers_ended is empty!")


offers_ended is empty!


In [ ]:
snowflake_query("egypt", "")